In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from sklearn.decomposition import PCA
import pickle

import sys
print(sys.path.append("/data/hdd1/brain/BraTS19/YandexCup/CLIP"))
import clip
from Visual_embeddings import VisualEmb

None


In [20]:
model, _ = clip.load("RN50")

In [50]:
df = pd.read_csv("clasters_data/BOW2_3mln.csv")
data = df.Text.to_list()

In [49]:
# token convert to embedding
data_token = clip.tokenize(data)
bs = 4
dataset = DataLoader(data_token, batch_size=bs)

embedings = torch.zeros((data_token.shape[0],4,768))
step = [0, 0]
for sents in dataset:
    sents_g = sents.cuda()
    step[1] += bs
    embedings[step[0]:step[1]] = model.bert.embeddings(sents_g).cpu().detach()
    step[0] += bs
    del sents_g
embedings.shape

100%|██████████| 20000/20000 [00:01<00:00, 19835.34it/s]


torch.Size([20000, 4, 768])

In [51]:
n = embedings.shape[0]
n_components = 400
pca_model = PCA(n_components = n_components, random_state=32)

main_class_emb = pca_model.fit_transform(embedings.reshape(n,-1))

In [35]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=40, random_state=0)
kmeans.fit(main_class_emb)

KMeans(n_clusters=40, random_state=0)

In [36]:
with open("kmeansv2.pkl", "wb") as f:
    pickle.dump(kmeans, f)

In [5]:
### second part


from typing import List, Tuple
import sys
print(sys.path.append("/data/hdd1/brain/BraTS19/YandexCup/sentence-transformers"))
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch

None


In [227]:
# clastering BOW top N elements -> and learning KNeighborsClassifier 

In [3]:
df = pd.read_csv("clasters_data/BOW2_3mln.csv")
data = df.Text.to_numpy()

In [6]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np


embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences

corpus_embeddings = embedder.encode(data)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)
print(f"Completed compute emb")

Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(data[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

Completed compute emb


In [231]:
###save inform about clasters
from itertools import chain 

In [237]:
k = list(chain.from_iterable([[(sent,label) for sent in sents]for label, sents in clustered_sentences.items()]))

In [240]:
pd.DataFrame.from_dict({"Sent":[i[0] for i in k],
                       "Calster":[i[1] for i in k],
                       }).to_csv("ClastersInform.csv",index = False)

In [72]:
###create model
from sklearn.neighbors import KNeighborsClassifier

In [161]:
def clasters_to_dataclass(clustered_s):
    X = np.zeros((20000,384)) #feature
    y = np.zeros(20000, dtype=np.int32) # clasters
    step = [0, 0]
    for ind in set(clustered_s.keys()):
        corpus_embeddings = embedder.encode(clustered_s[ind])
        corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True) 
        step[1] += corpus_embeddings.shape[0]
        X[step[0]:step[1]] = corpus_embeddings
        y[step[0]:step[1]] = ind
        step[0] += corpus_embeddings.shape[0]
    return X, y

In [162]:
X,y = clasters_to_dataclass(clustered_sentences)

['на рабочий стол', 'обои на рабочий', 'обои для рабочий', 'рабочий стол на', 'рабочий стол эстетика', 'обои рабочий стол', 'рабочий стол обои', 'рабочий стол скачать', 'рабочий стол из', 'рабочий стол стиль', 'рабочий стол пк', 'рабочий стол ноутбук']


In [222]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
import pickle

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle=True,random_state=42)

In [220]:
model = KNeighborsClassifier(n_neighbors = 6) # 6-7
model.fit(X_train,y_train)

pr = model.predict(X_test)
accuracy_score(pr,y_test)

0.6276

In [226]:
pickle.dump(model,open("KnnClassif.pkl","wb"))